<img src="http://www.redage.org/sites/default/files/styles/large/public/img-logo-institucion/logo_iteso.jpg?itok=IzloBJS1">

# Ingeniería Financiera
## ITE1731 - Microestructura y Sistemas de Trading
## IF Sophia Ortega Lares
### Septiembre 2020|Repositorio: https://github.com/if708925/myst_if708925_lab1

## Abstract
#### Este documento fue elaborado por Sophia Ortega Lares como un trabajo parcial para la materia de Microestructura y Sistemas de Trading - ITE1731, la cual es parte del currículum de la licenciatura en Ingeniería Financiera, ofertada por la universidad ITESO. En el presente trabajo se plantea la respuesta a la siguiente pregunta: ¿Qué estrategia de inversión propondrías si tu trabajo fuera administrar 1 millón de pesos?

# Laboratorio 1
#### Inversión de Capital

# 0. Librerías y Dependencias
## 0.1 Librerías
##### Para correr este Notebook es necesario tener instaladas y/o en el archivo requirements.txt las siguientes librerias:
##### pandas>=1.1.0
##### numpy>=1.19.1
##### jupyter>=1.0.0
##### yfinance>=0.1.54
##### matplotlib>=3.3.1

## 0.2 Dependencias
##### Para correr este Notebook es necesario contar con los siguientes archivos y dependencias externas:
##### files/precios.csv : Precios históricos
##### files/datos.txt : Otros datos históricos


# 1. Introducción
#### A menudo los financieros se topan con el dilema de: ¿que tipo de estrategia de inversión utilizar para admnistrar portafolios de inversión? Es muy complicado encontrar el balance entre riesgo, utilidad y costo. Y dependiendo de l ambiente en el que se desarrolle hay mas variables que se tienen que tomar en cuenta a la hora de la toma de decisiones, por esto elegir entre una estrategia pasiva y una activa puede ser un gran reto.
#### Claro que cada estrategia tiene ventajas y desventajas, una estrategia pasiva tiende a hacer más sencillo el trabajo de los administradores de portafolios, y si esto sucede, ocasiona una disminución en los márgenes al tener que competir por los precios.
#### Por otro lado una estrategia pasiva en mercados emergentes puede ser una buena opción, pues los costos de transacción tienden a ser muy elevados para optar por una estrategia de inversión activa, además de que el costo de tomar malas decisiones al elegir al administrador es más elevado.
#### Pero hay una cuestión muy importante a tomar en cuenta al elegir la estrategia pasiva: no hay una disciplina de evaluación, el anadir una acción al índice no depende de ningún tipo de evaluación, y esto a menudo puede ser peligroso si hay acciones en el índice que están infladas.
#### En cambio, la estrategia activa requiere un buen manejo de la volatilidad, de manera que el administrador pueda adaptar su estrategia a sacarle provecho a los cambios en el mercado, precisamente diversificando el portafolio y comprando y vendiendo cada que se llega al objetivo. 
#### Pero esto tambien puede significar una desventaja al trabajar en contra de la oportunidad de que un sector individual te puede dar mas utilidad que el mercado en general, en otras palabras: la inversión activa podría significar una falta de paciencia de administradores que se concentran solo en utilidades relativas a un “benchmark”, en vez de utilidades absolutas, y que además no protegen el capital del cliente al trabajar con una estrategia que consiste en solo seguir al mercado


# 2. Objetivo
#### El objetivo del primer laboratorio consiste en entender la diferencia entre la inversión activa y la inversión pasiva, se pretende realizar 2 series de tiempo: La primer serie de tiempo representando la evolución del capital si se realizara una estrategia de inversión pasiva, y la segunda representa la evolución del capital de la estrategia de Inversión Activa.


# 3.Datos utilizados


# 4.Procesos


In [6]:
import os
import numpy as np
import pandas as pd
import time
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
# 1.1 Obtener la lista de los archivos a leer data.py
#%%
path = '/Users/Catalina/Documents/ITESO/MST/myst_if708925_lab1/files/NAFTRAC_holdings'
abspath = os.path.abspath(path)
archivos = [f[:-4] for f in os.listdir(abspath) if os.path.isfile(os.path.join(abspath, f))]

In [7]:
# 1.2 leer todos los archivos y guardarlos en un diccionario functions.py
data_archivos = {}
for i in archivos:
    # i = achivos[0]
    # leer archivos despues de los primeros 2 renglones
    data = pd.read_csv('files/NAFTRAC_holdings/' + i + '.csv', skiprows=2, header=None)
    # renombrar columnas
    data.columns = list(data.iloc[0, :])
    data = data.loc[:, pd.notnull(data.columns)]
    data = data.iloc[1:-1].reset_index(drop=True, inplace=False)
    # quitar comas
    data['Precio'] = [i.replace(',','') for i in data['Precio']]
    # quitar asteriscos
    data['Ticker'] = [i.replace('*','') for i in data['Ticker']]
    convert_dict = {'Ticker': str, 'Nombre': str, 'Peso (%)': float, 'Precio': float}
    data = data.astype(convert_dict)
    # convertir a decimal la columna de peso
    data['Peso (%)'] = data['Peso (%)']/100
    # guardar en diccionario
    data_archivos[i] = data

In [8]:
# 1.3 Construir el vector de fechas a partir del vector de nombres de archivos functions.py
# sirve como etiqueta en dataframe y para yfinance
t_fechas = [i.strftime('%d-%m-%Y') for i in sorted([pd.to_datetime(i[8:]).date() for i in archivos])]
#lista con fechas ordenadas para usarse como indexadores de archivos
i_fechas = [j.strftime('%Y-%m-%d') for j in sorted([pd.to_datetime(i[8:]).date() for i in archivos])]

In [9]:
# 1.4 Construir el vector de tickers utilizables en yahoo finance functions.py
# -- descargar y acomodar los datos
ticker = []
for i in archivos:
    # i = archivos[1]
    l_tickers = list(data_archivos[i]['Ticker'])
    print(l_tickers)
    [ticker.append(i + '.MX') for i in l_tickers]
#lista global (todos los datos que vamos a necesitar para descargar)
global_tickers = np.unique(ticker).tolist()

['AMXL', 'WALMEX', 'FEMSAUBD', 'GFNORTEO', 'GMEXICOB', 'CEMEXCPO', 'TLEVISACPO', 'ELEKTRA', 'GAPB', 'BIMBOA', 'GRUMAB', 'KIMBERA', 'KOFUBL', 'ASURB', 'AC', 'PE&OLES', 'ALFAA', 'ORBIA', 'GFINBURO', 'PINFRA', 'OMAB', 'IENOVA', 'BOLSAA', 'CUERVO', 'GCARSOA1', 'MEGACPO', 'VESTA', 'LABB', 'Q', 'BBAJIOO', 'ALSEA', 'LIVEPOLC.1', 'BSMXB', 'GENTERA', 'MXN', 'RA']
['AMXL', 'FEMSAUBD', 'GFNORTEO', 'GMEXICOB', 'WALMEX', 'CEMEXCPO', 'TLEVISACPO', 'ALFAA', 'GAPB', 'ASURB', 'KOFL', 'MEXCHEM', 'BIMBOA', 'AC', 'KIMBERA', 'GFINBURO', 'IENOVA', 'GRUMAB', 'BSMXB', 'PINFRA', 'PE&OLES', 'ELEKTRA', 'OMAB', 'ALSEA', 'GCARSOA1', 'MEGACPO', 'LIVEPOLC.1', 'CUERVO', 'GENTERA', 'VOLARA', 'LALAB', 'LABB', 'GFREGIOO', 'ALPEKA', 'NEMAKA', 'MXN']
['FEMSAUBD', 'AMXL', 'GFNORTEO', 'WALMEX', 'GMEXICOB', 'CEMEXCPO', 'TLEVISACPO', 'BIMBOA', 'ALFAA', 'ASURB', 'GAPB', 'GFINBURO', 'KOFL', 'ELEKTRA', 'MEXCHEM', 'BSMXB', 'KIMBERA', 'GRUMAB', 'AC', 'PINFRA', 'IENOVA', 'OMAB', 'PE&OLES', 'MEGACPO', 'BBAJIOO', 'ALSEA', 'LIVEPOLC.1

In [10]:
# 1.5 Obtener posiciones historicas main.py
global_tickers = [i.replace('GFREGIOO.MX', 'RA.MX') for i in global_tickers]
global_tickers = [i.replace('MEXCHEM.MX', 'ORBIA.MX') for i in global_tickers]
global_tickers = [i.replace('LIVEPOLC.1.MX', 'LIVEPOLC-1.MX') for i in global_tickers]

# ELIMINAR MXN, USD, KOFL y usar como cash
#cuando se utiliza KOF, ese % o ponderacion se tiene que pasar a CASH
[global_tickers.remove(i) for i in ['MXN.MX', 'USD.MX', 'KOFL.MX', 'KOFUBL.MX', 'BSMXB.MX']]
# contar el tiempo que tarda
inicio = time.time()
# descarga de yahoofinance
data = yf.download(global_tickers, start="2018-01-31", end="2020-08-25", actions=False,
                   group_by="close", interval='1d', auto_adjust=False, prepost=False, threads=True)

print('se tardo', round(time.time() - inicio, 2), 'segundos')
#convertir columna de fechas de cierre en un dataframe
data_close = pd.DataFrame({i: data[i]['Close'] for i in global_tickers})

# tomar solo las fechas de interes y ponerlas en una lista
ic_fechas = sorted(list(set(data_close.index.astype(str).tolist()) & set(i_fechas)))

#localizar todos los precios (encontrar todos los precios de cada mes)
precios = data_close.iloc[[int(np.where(data_close.index.astype(str) == i)[0]) for i in ic_fechas]]

# ordenar columnas lexicograficamente
precios = precios.reindex(sorted(precios.columns), axis=1)

# tomar solo las columnas de interes
# transponer matriz para tener x: fechas, y: precios
# multiplicar matriz de precios por matriz de pesos
# hacer suma de cada columna para obtener valor de mercado

[*********************100%***********************]  39 of 39 completed
se tardo 21.3 segundos


In [11]:
# 1.6 posicion inicial main.py
#capital inicial
k = 1000000
#comision
c = 0.00125
#vector de comisiones historicas
comisiones = []

# obtener posicion inicial, los % para KOFL, KOFUBL, BSMXB, USD asignarlos a cash
#quitar tickers cuando aparezcan. la suma de lo que te gastaste en hace las posiciones de el resto
#entonces la diferencia es el CASH
c_activos = ['KOFL', 'KOFUBL', 'BSMXB', 'MXN', 'USD']

#diccionario para resultado final
inv_pasiva = {'timestamp': t_fechas, 'capital': [k]}

In [13]:
# ------------------------------------------------------------------------------------------------------------------
# 1.7 Evolucion de la postura (Inversion pasiva) visualization.py

pos_datos = data_archivos[archivos[0]].copy().sort_values('Ticker')[['Ticker', 'Nombre', 'Peso (%)']]
# extraer la lista de activos a eliminar
i_activos = list(pos_datos[list(pos_datos['Ticker'].isin(c_activos))].index)
# eliminar los activos del dataframe
pos_datos.drop(i_activos, inplace=True)
# resetear el index
pos_datos.reset_index(inplace=True, drop=True)
# agregar .MX para empatar precios
pos_datos['Ticker'] = pos_datos['Ticker'] + '.MX'
#corregir tickers en datos
pos_datos['Ticker'] = pos_datos['Ticker'].replace('LIVEPOLC.1.MX', 'LIVEPOLC-1.MX')
pos_datos['Ticker'] = pos_datos['Ticker'].replace('MEXCHEM.MX', 'ORBIA.MX')
pos_datos['Ticker'] = pos_datos['Ticker'].replace('GFREGIOO.MX', 'RA.MX')
# fecha para la que se busca hacer el match de precios
match = 0
# precios necesarios para la posicion
pos_datos['Precio'] = [precios.iloc[match, precios.columns.to_list().index(i)] for i in pos_datos['Ticker']]
pos_datos['Capital'] = pos_datos['Peso (%)'] * k - pos_datos['Peso (%)'] * k * c

# cantidad de titulos por accion (solo se hace una vez)
pos_datos['Titulos'] = pos_datos['Capital'] // pos_datos['Precio']
# valor de la postura por accion (solo se hace una vez)
pos_datos['Postura'] = pos_datos['Titulos'] * pos_datos['Precio']
# comision pagada (solo se hace una vez)
pos_datos['Comision'] = pos_datos['Postura'] * c

pos_comision = pos_datos['Comision'].sum()
pos_cash = k - pos_datos['Postura'].sum() - pos_comision
#suma de todas las posiciones
pos_value = pos_datos['Postura'].sum()
capital = pos_value + pos_cash

for i in archivos:
    i = archivos.index(i)
    match = i
    precios.index.to_list()[match]
    # precios necesarios para la posicion
    m2 = [precios.iloc[match, precios.columns.to_list().index(i)] for i in pos_datos['Ticker']]
    pos_datos['Precio'] = m2
    #Postura, precio por cantidad de titulos
    pos_datos['Postura'] = pos_datos['Titulos'] * pos_datos['Precio']
    #suma de todas las posiciones
    pos_value = pos_datos['Postura'].sum()
    inv_pasiva['capital'].append(pos_value + pos_cash)

df_pasiva = pd.DataFrame.from_dict(inv_pasiva, orient='index').transpose()
df_pasiva.set_index('timestamp', inplace=True)
df_pasiva['rend'] = df_pasiva.pct_change()
df_pasiva['rend_acum'] = np.cumsum(df_pasiva['rend'])
df_pasiva = df_pasiva.reset_index()


## DataFrame de la evolución de la inversión pasiva


In [15]:
print(df_pasiva)

     timestamp  capital      rend  rend_acum
0   31-01-2018  1000000       NaN        NaN
1   28-02-2018   998788 -0.001212  -0.001212
2   28-03-2018   950932 -0.047914  -0.049126
3   30-04-2018   930743 -0.021231  -0.070357
4   31-05-2018   976595  0.049264  -0.021093
..         ...      ...       ...        ...
60         NaN   842601  0.064903  -0.096395
61         NaN   828986 -0.016159  -0.112554
62         NaN   859937  0.037336  -0.075218
63         NaN   844628 -0.017802  -0.093020
64         NaN   868668  0.028462  -0.064558

[65 rows x 4 columns]


## 5. Medidas de atribucion al desempeno (rendimiento mensual promedio, rendimiento mensual acumulado, sharpe)

In [16]:
rend_pm = df_pasiva['rend'].mean()
rendacumm = df_pasiva['rend'].sum()
sharper = rend_pm/df_pasiva['rend'].std()
medidas = [['Rendimiento mensual promedio', rend_pm], ['Rendimiento mensual acumulado', rendacumm], ['Sharpe ratio', sharper]]
df_medidas = pd.DataFrame(medidas, columns=['descripcion', 'inv_pasiva'])
print(df_medidas)

                     descripcion  inv_pasiva
0   Rendimiento mensual promedio   -0.001009
1  Rendimiento mensual acumulado   -0.064558
2                   Sharpe ratio   -0.020670


# 5. Conclusión
#### A partir de este análisis pude conocer a detalle cómo se desarrolla cada tipo de estrategia, lo que implica realizar cada una y lo que implica en el rendimiento final, en la inversión pasiva me di cuenta de lo mucho que importa seleccionar las ponderaciones apropiadas al inicio de la inversión. Y lo mucho que te puedes ahorrar en comisiones. Por otro lado, en la inversión activa a pesar de no haberla representado en el trabajo final, pude entender el reto que significa estar más atento con los cambios diarios del precio y como sacrificas una parte de la ganancia por pagar costos de comisión. 
#### Al final, como financieros es muy importante que la meta final sea obtener el mayor rendimiento al menor riesgo, pero muchas veces no nos damos cuenta de que hay otros factores que se deben de tomar en cuenta al tomar decisiones, sobre todo al invertir dinero que no nos pertenece, como seguro lo estaremos haciendo muchos de nosotros.
#### Tenemos una gran responsabilidad de darle un valor añadido a los clientes, no considero muy positivo solo enfocarse en un benchmark dejando de lado rendimientos absolutos en un sector determinado, vender cuando el mercado va mal y comprar cuando el mercado va bien es hacer lo que todos están haciendo. 
#### Entonces lo mejor que se puede hacer es ser un administrador que pueda abarcar muchos sectores y encontrar oportunidades de inversión. Porque la volatilidad a pesar de que muchas veces nos hace perder mucho dinero, sin ella no podríamos ganar tampoco, es un factor inevitable, incontrolable e impredecible, entonces si somos un poco más críticos a la hora de invertir el dinero, seguro que nos vamos a posicionar muchísimo mejor si pensamos en añadir valor más que en seguir al mercado.
